## pipeline

In [1]:
import networkx as nx
import numpy as np
import torch

import torch.nn as nn

from statistics import mean

from func.run_pipeline_super_vox import get_outlayer_of_a_3d_shape, get_crop_by_pixel_val
from func.ultis import load_obj

from func.graph_learning import SuperVoxToNxGraph, VoxelGraphDataset


Using backend: pytorch


In [2]:
# load graphs
from func.ultis import load_obj

graphs = load_obj("graphs_dataset_train")


In [3]:
dataset = VoxelGraphDataset(graphs)

g = dataset[0]


# TODO probably should normalize features!!!!

In [4]:
from dgl.dataloading import GraphDataLoader
from torch.utils.data.sampler import SubsetRandomSampler

num_examples = len(dataset)
num_train = int(num_examples)

train_sampler = SubsetRandomSampler(torch.arange(num_train))
test_sampler = SubsetRandomSampler(torch.arange(num_train, num_examples))

train_dataloader = GraphDataLoader(
    dataset, sampler=train_sampler, batch_size=5, drop_last=False)

In [12]:
from func.graph_models import GCN, GCN_2
import torch.nn.functional as F
import random

model = GCN(1, num_classes=2)
CELoss = nn.CrossEntropyLoss()

# training
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
best_val_acc = 0



# features = g.ndata['feat']
# labels = g.ndata['label']
# train_mask = g.ndata['train_mask']
# val_mask = g.ndata['val_mask']

# calculate weights for loss
"""
pos_weights = []
neg_weights = []
for graph_number in range(len(dataset)):
    sample_graph = dataset[graph_number]
    labels = sample_graph.ndata['label']
    # create class weights
    number_positives = torch.count_nonzero(labels)
    positive_weight = 1 - (number_positives / len(labels))
    negative_weight = 1 - positive_weight

    pos_weights.append(positive_weight.item())
    neg_weights.append(negative_weight.item())
weights = torch.tensor([mean(neg_weights), mean(pos_weights)])
print(f"weights: {weights}")
"""
epoch_loss = []
epoch_accuracy = []

epoch_accuracy_val = []
for e in range(2000):
    # get random elements for batch
    #graphs_numbers_list = range(0, len(dataset))
    #rand_graph_numbers = random.sample(graphs_numbers_list, len(dataset))
    for graph_number in range(len(dataset)):
    #for graph_number in range(1):
    #for graph_number in rand_graph_numbers:
        # Forward
        sample_graph = dataset[graph_number]
        features = sample_graph.ndata['feat']
        labels = sample_graph.ndata['label']
        train_mask = sample_graph.ndata['train_mask']
        val_mask = sample_graph.ndata['val_mask']

        # create class weights
        number_positives = torch.count_nonzero(labels)
        percentage_positives = number_positives / len(labels)
        percentage_negatives = 1 - percentage_positives

        weights = torch.tensor([1 - percentage_negatives, 1 - percentage_positives])
        #weights = torch.tensor([0.95, 0.05])
        #print(weights)

        CELoss = nn.CrossEntropyLoss(weight=weights)
        train_mask = sample_graph.ndata['train_mask']
        val_mask = sample_graph.ndata['val_mask']
        logits = model(sample_graph, features)

        # Compute prediction
        pred = logits.argmax(1)

        # Compute loss
        # Note that you should only compute the losses of the nodes in the training set.
        loss = CELoss(logits[train_mask], labels[train_mask])
        epoch_loss.append(loss.item())
        #print(loss)
        # Compute accuracy on training/validation/test
        train_acc = (pred[train_mask] == labels[train_mask]).float().mean()
        val_acc = (pred[val_mask] == labels[val_mask]).float().mean()
        epoch_accuracy.append(train_acc.item())
        epoch_accuracy_val.append(val_acc.item())
        # Backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

    if e % 20 == 0:
        print('In epoch {}, loss: {:.3f}, accuracy: {:.3f}, val accuracy: {:.3f}'.format(
            e, mean(epoch_loss), mean(epoch_accuracy), mean(epoch_accuracy_val)))
        epoch_loss = []
        epoch_accuracy = []


/opt/anaconda3/envs/CellSeg3D_torch/lib/python3.8/site-packages/torch/autocast_mode.py:162: UserWarning: User provided device_type of 'cuda', but CUDA is not available. Disabling
  warnings.warn('User provided device_type of \'cuda\', but CUDA is not available. Disabling')


In epoch 0, loss: 0.715, accuracy: 0.464, val accuracy: 0.470
In epoch 20, loss: 0.692, accuracy: 0.896, val accuracy: 0.858
In epoch 40, loss: 0.692, accuracy: 0.934, val accuracy: 0.884
In epoch 60, loss: 0.692, accuracy: 0.933, val accuracy: 0.893
In epoch 80, loss: 0.692, accuracy: 0.930, val accuracy: 0.896
In epoch 100, loss: 0.692, accuracy: 0.899, val accuracy: 0.893
In epoch 120, loss: 0.692, accuracy: 0.885, val accuracy: 0.888
In epoch 140, loss: 0.692, accuracy: 0.883, val accuracy: 0.885
In epoch 160, loss: 0.692, accuracy: 0.835, val accuracy: 0.877
In epoch 180, loss: 0.692, accuracy: 0.825, val accuracy: 0.869
In epoch 200, loss: 0.692, accuracy: 0.829, val accuracy: 0.864
In epoch 220, loss: 0.692, accuracy: 0.828, val accuracy: 0.859
In epoch 240, loss: 0.692, accuracy: 0.821, val accuracy: 0.855
In epoch 260, loss: 0.692, accuracy: 0.832, val accuracy: 0.852
In epoch 280, loss: 0.692, accuracy: 0.835, val accuracy: 0.849
In epoch 300, loss: 0.691, accuracy: 0.822, va

KeyboardInterrupt: 

from func.graph_models import GCN, GCN_2
import torch.nn.functional as F
import random
import dgl

model = GCN(1, num_classes=2)
CELoss = nn.CrossEntropyLoss()

# training
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
best_val_acc = 0



# features = g.ndata['feat']
# labels = g.ndata['label']
# train_mask = g.ndata['train_mask']
# val_mask = g.ndata['val_mask']

# calculate weights for loss
"""
pos_weights = []
neg_weights = []
for graph_number in range(len(dataset)):
    sample_graph = dataset[graph_number]
    labels = sample_graph.ndata['label']
    # create class weights
    number_positives = torch.count_nonzero(labels)
    positive_weight = 1 - (number_positives / len(labels))
    negative_weight = 1 - positive_weight

    pos_weights.append(positive_weight.item())
    neg_weights.append(negative_weight.item())
weights = torch.tensor([mean(neg_weights), mean(pos_weights)])
print(f"weights: {weights}")
"""

# build one big graph
graphs_list = []
for i in range(len(dataset)):
    graphs_list.append(dataset[i])

large_g = dgl.batch(graphs_list)

epoch_loss = []
epoch_accuracy = []
for e in range(1000):
    # get random elements for batch
    #graphs_numbers_list = range(0, len(dataset))
    #rand_graph_numbers = random.sample(graphs_numbers_list, len(dataset))
    sample_graph = large_g
    features = sample_graph.ndata['feat']
    labels = sample_graph.ndata['label']

    # create class weights
    number_positives = torch.count_nonzero(labels)
    percentage_positives = number_positives / len(labels)
    percentage_negatives = 1 - percentage_positives

    weights = torch.tensor([1 - percentage_negatives, 1 - percentage_positives])
    #weights = torch.tensor([0.95, 0.05])
    #print(weights)

    CELoss = nn.CrossEntropyLoss(weight=weights)
    #train_mask = sample_graph.ndata['train_mask']
    #val_mask = sample_graph.ndata['val_mask']
    logits = model(sample_graph, features)

    # Compute prediction
    pred = logits.argmax(1)

    # Compute loss
    # Note that you should only compute the losses of the nodes in the training set.
    loss = CELoss(logits, labels)
    epoch_loss.append(loss.item())
    #print(loss)
    # Compute accuracy on training/validation/test
    train_acc = (pred == labels).float().mean()
    epoch_accuracy.append(train_acc.item())
    # Backward
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    if e % 20 == 0:
        print('In epoch {}, loss: {:.3f}, accuracy: {:.3f}'.format(
            e, mean(epoch_loss), mean(epoch_accuracy)))
        epoch_loss = []
        epoch_accuracy = []




# Train on GPU
g = g.to('cuda')
model = GCN(1, 16, dataset.num_classes).to('cuda')
train(g, model)

In [13]:
g

Graph(num_nodes=1405, num_edges=5327,
      ndata_schemes={'feat': Scheme(shape=(1,), dtype=torch.float32), 'label': Scheme(shape=(), dtype=torch.int64), 'train_mask': Scheme(shape=(), dtype=torch.bool), 'val_mask': Scheme(shape=(), dtype=torch.bool)}
      edata_schemes={'weight': Scheme(shape=(1,), dtype=torch.float64)})

In [14]:

g.ndata['feat'].shape

torch.Size([1405, 1])

In [15]:

predictions = model(g, g.ndata['feat']).argmax(1).numpy()

In [16]:
np.unique(predictions)

array([0, 1])

In [ ]:
labels = g.ndata['label']
np.unique(labels)

In [ ]:
len(labels[labels==1])

In [ ]:
len(labels[labels==0])

In [ ]:
predictions